In [2]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from data.custom_dataset import CustomDataset
from Network.conditional_net import ConditionalNet
from Network.sdf_net import SDFNet

In [3]:
MACRO_SIZE = 124
LSTM_HIDDEN = 16
LSTM_LAYER = 1
CHAR_SIZE = 27
FF_HIDDEN = [32, 16]
FF_LAYER = 2
BATCH_SIZE = 3

In [4]:
conditional_net = ConditionalNet(
    MACRO_SIZE, CHAR_SIZE, LSTM_HIDDEN, LSTM_LAYER, FF_HIDDEN, FF_LAYER)
sdf_net = SDFNet(
    MACRO_SIZE, CHAR_SIZE, LSTM_HIDDEN, LSTM_LAYER, FF_HIDDEN, FF_LAYER)
optimizer_C = torch.optim.Adam(conditional_net.parameters(), lr=0.0002, betas=(0.9, 0.99))
optimizer_S = torch.optim.Adam(sdf_net.parameters(), lr=0.1, betas=(0.9, 0.99))

In [5]:
CHAR_PATH = "./data/27_features_rets_normalized_filled.csv"
MACRO_PATH = "./data/124_macro_data.csv"
dataset = CustomDataset(CHAR_PATH, MACRO_PATH)

In [9]:
for i in range(len(dataset)):
    data = dataset[i]
    char = torch.from_numpy(data["char"]).float()
    macro = torch.from_numpy(data["macro"]).float()
    ret = torch.from_numpy(data["ret"]).float()
    
    SDF = sdf_net(macro, char, ret)
    g_hat = conditional_net(macro,char,ret)
    conditional_loss = -1*(torch.sum((SDF*g_hat).pow(2)))
    optimizer_C.zero_grad()
    conditional_loss.backward()
    optimizer_C.step()


    for j in range(50):
        SDF = sdf_net(macro, char, ret)
        g_hat = conditional_net(macro,char,ret)
        sdf_loss = (torch.sum((SDF*g_hat).pow(2)))
        g_hat_mean = torch.mean(g_hat)

        optimizer_S.zero_grad()
        sdf_loss.backward()
        optimizer_S.step()
    print("[Iter %d/%d]  [sdf_loss: %f] [sdf: %f] [g_hat_mean: %f]"
                % (i, len(dataset), sdf_loss.item(), SDF.item(), g_hat_mean.item())
                )
    SDF = sdf_net(macro, char, ret)
    factor_price = 1-SDF
    beta = ret/factor_price
    print(factor_price)
    print(beta)

[Iter 0/659]  [sdf_loss: 0.001401] [sdf: -0.002890] [g_hat_mean: 0.002241]
tensor(1.0191, grad_fn=<RsubBackward1>)
tensor([-0.6074,  0.1398,  0.1457,  ..., -0.8514, -0.6677, -0.2401],
       grad_fn=<DivBackward0>)
[Iter 1/659]  [sdf_loss: 0.055866] [sdf: -0.019415] [g_hat_mean: 0.003151]
tensor(0.9908, grad_fn=<RsubBackward1>)
tensor([ 0.2534, -0.1563, -0.2226,  ..., -0.7733,  1.5218,  0.4607],
       grad_fn=<DivBackward0>)
[Iter 2/659]  [sdf_loss: 0.013809] [sdf: -0.009682] [g_hat_mean: 0.004308]
tensor(1.0064, grad_fn=<RsubBackward1>)
tensor([ 0.3769,  0.0270,  0.0215,  ...,  0.6264, -1.3830,  1.6544],
       grad_fn=<DivBackward0>)
[Iter 3/659]  [sdf_loss: 0.004116] [sdf: 0.005446] [g_hat_mean: 0.001639]
tensor(0.9978, grad_fn=<RsubBackward1>)
tensor([-0.4420, -0.9522, -0.2799,  ..., -0.6110, -0.0537, -0.6586],
       grad_fn=<DivBackward0>)
[Iter 4/659]  [sdf_loss: 0.096755] [sdf: -0.023933] [g_hat_mean: -0.009260]
tensor(1.0252, grad_fn=<RsubBackward1>)
tensor([-0.3035,  0.5346,

[Iter 38/659]  [sdf_loss: 0.193763] [sdf: -0.030565] [g_hat_mean: 0.009972]
tensor(1.1858, grad_fn=<RsubBackward1>)
tensor([ 0.3585,  0.1081,  0.3359,  ..., -0.7370, -1.3958, -0.1092],
       grad_fn=<DivBackward0>)
[Iter 39/659]  [sdf_loss: 140.478424] [sdf: 0.814881] [g_hat_mean: 0.013079]
tensor(0.1852, grad_fn=<RsubBackward1>)
tensor([-4.1668,  3.9666, -3.2419,  ..., -4.0531, 18.3228,  2.1337],
       grad_fn=<DivBackward0>)
[Iter 40/659]  [sdf_loss: 160.137939] [sdf: 0.798468] [g_hat_mean: -0.001395]
tensor(0.2016, grad_fn=<RsubBackward1>)
tensor([12.9816, -5.2881,  2.0027,  ..., -0.6509, -2.4488, -1.4633],
       grad_fn=<DivBackward0>)
[Iter 41/659]  [sdf_loss: 227.569168] [sdf: 1.052225] [g_hat_mean: 0.005731]
tensor(-0.0521, grad_fn=<RsubBackward1>)
tensor([-12.9927, -13.6941,  12.8198,  ...,  27.0114,  -2.8247, -11.8044],
       grad_fn=<DivBackward0>)
[Iter 42/659]  [sdf_loss: 221.986099] [sdf: 0.998357] [g_hat_mean: 0.006522]
tensor(0.0018, grad_fn=<RsubBackward1>)
tensor([

[Iter 76/659]  [sdf_loss: 464.903229] [sdf: 1.156434] [g_hat_mean: -0.010774]
tensor(-0.1564, grad_fn=<RsubBackward1>)
tensor([-0.8329,  3.3296, -1.6585,  ..., -4.2339, -5.3963,  4.6190],
       grad_fn=<DivBackward0>)
[Iter 77/659]  [sdf_loss: 379.777191] [sdf: 1.038837] [g_hat_mean: -0.019416]
tensor(-0.0388, grad_fn=<RsubBackward1>)
tensor([ 10.7492, -36.4751, -14.9660,  ...,  -0.1144,  39.7039,  -1.1461],
       grad_fn=<DivBackward0>)
[Iter 78/659]  [sdf_loss: 333.165955] [sdf: 0.969767] [g_hat_mean: 0.001242]
tensor(0.0303, grad_fn=<RsubBackward1>)
tensor([ 39.7895, -45.3402,  28.1105,  ..., -13.7592,  18.8745,  43.1507],
       grad_fn=<DivBackward0>)
[Iter 79/659]  [sdf_loss: 60.707199] [sdf: 0.406061] [g_hat_mean: -0.002302]
tensor(0.6093, grad_fn=<RsubBackward1>)
tensor([ 0.0069, -1.7078, -0.4600,  ...,  2.5369,  0.5667, -0.9083],
       grad_fn=<DivBackward0>)
[Iter 80/659]  [sdf_loss: 352.687805] [sdf: 0.954589] [g_hat_mean: -0.007515]
tensor(0.0455, grad_fn=<RsubBackward1>

[Iter 113/659]  [sdf_loss: 880.682678] [sdf: 0.981896] [g_hat_mean: -0.001299]
tensor(0.0187, grad_fn=<RsubBackward1>)
tensor([-48.0609,  59.6923,  -3.6434,  ..., -12.6913, -87.8592, -53.1414],
       grad_fn=<DivBackward0>)
[Iter 114/659]  [sdf_loss: 851.906311] [sdf: 0.934366] [g_hat_mean: -0.000856]
tensor(0.0664, grad_fn=<RsubBackward1>)
tensor([-8.2848, -8.4614, 10.0763,  ..., -0.8778,  4.5363, 10.0763],
       grad_fn=<DivBackward0>)
[Iter 115/659]  [sdf_loss: 724.548279] [sdf: 0.865391] [g_hat_mean: -0.002020]
tensor(0.1365, grad_fn=<RsubBackward1>)
tensor([  6.4863,   4.6400,  -0.4282,  ..., -15.0698, -17.0713, -19.4944],
       grad_fn=<DivBackward0>)
[Iter 116/659]  [sdf_loss: 731.090820] [sdf: 0.916962] [g_hat_mean: 0.005285]
tensor(0.0858, grad_fn=<RsubBackward1>)
tensor([  3.3165,  -0.1962,  -3.3562,  ..., -25.0813,  -5.5287,  54.2793],
       grad_fn=<DivBackward0>)
[Iter 117/659]  [sdf_loss: 451.110626] [sdf: 0.669235] [g_hat_mean: -0.007816]
tensor(0.3353, grad_fn=<Rsub

tensor(0.1719, grad_fn=<RsubBackward1>)
tensor([-1.7220, 11.3784, -0.2121,  ..., -4.9654, -2.0907, 11.5206],
       grad_fn=<DivBackward0>)
[Iter 151/659]  [sdf_loss: 1076.546387] [sdf: 0.806991] [g_hat_mean: -0.000017]
tensor(0.1966, grad_fn=<RsubBackward1>)
tensor([ 1.8265,  0.0244, -1.4873,  ..., 14.3404, -5.7951, -2.3725],
       grad_fn=<DivBackward0>)
[Iter 152/659]  [sdf_loss: 604.121887] [sdf: 0.581796] [g_hat_mean: -0.002681]
tensor(0.4256, grad_fn=<RsubBackward1>)
tensor([-0.2511,  1.2174, -1.5593,  ..., -2.7435, -0.8926, -7.1121],
       grad_fn=<DivBackward0>)
[Iter 153/659]  [sdf_loss: 50.944916] [sdf: 0.171524] [g_hat_mean: 0.006441]
tensor(0.9163, grad_fn=<RsubBackward1>)
tensor([-0.4514, -0.4270,  1.0633,  ...,  2.5010,  0.8785, -2.7999],
       grad_fn=<DivBackward0>)
[Iter 154/659]  [sdf_loss: 1.046989] [sdf: -0.024072] [g_hat_mean: 0.010373]
tensor(1.0243, grad_fn=<RsubBackward1>)
tensor([ 0.0024, -0.0669, -0.5956,  ..., -0.9191,  0.0760,  0.3412],
       grad_fn=<Di

...

[Iter 491/659]  [sdf_loss: 1807.861572] [sdf: 0.548308] [g_hat_mean: 0.025746]
tensor(0.4550, grad_fn=<RsubBackward1>)
tensor([-1.3799, -0.7075, -0.5603,  ..., -0.5610, -4.2229, -0.4428],
       grad_fn=<DivBackward0>)
[Iter 492/659]  [sdf_loss: 425.004089] [sdf: 0.285915] [g_hat_mean: -0.005454]
tensor(0.7177, grad_fn=<RsubBackward1>)
tensor([-0.1875,  0.7302,  0.3653,  ..., -0.0739, -0.8130, -0.8355],
       grad_fn=<DivBackward0>)
[Iter 493/659]  [sdf_loss: 373.634583] [sdf: 0.253622] [g_hat_mean: 0.021797]
tensor(0.7860, grad_fn=<RsubBackward1>)
tensor([ 1.2787, -0.3196,  1.8128,  ...,  0.5664,  0.1124, -0.6100],
       grad_fn=<DivBackward0>)
[Iter 494/659]  [sdf_loss: 13.367018] [sdf: -0.046540] [g_hat_mean: 0.002165]
tensor(1.0625, grad_fn=<RsubBackward1>)
tensor([-0.7553, -0.5528,  0.4595,  ..., -0.0401, -0.2267, -0.3974],
       grad_fn=<DivBackward0>)
[Iter 495/659]  [sdf_loss: 662.874390] [sdf: 0.342543] [g_hat_mean: -0.054671]
tensor(0.6653, grad_fn=<RsubBackward1>)
tensor(

[Iter 527/659]  [sdf_loss: 1741.858398] [sdf: 0.639728] [g_hat_mean: 0.046047]
tensor(0.3638, grad_fn=<RsubBackward1>)
tensor([ 0.7208, -3.5769, -1.4464,  ..., -0.4156, -4.5413,  1.4496],
       grad_fn=<DivBackward0>)
[Iter 528/659]  [sdf_loss: 2387.854248] [sdf: 0.832159] [g_hat_mean: -0.014116]
tensor(0.1719, grad_fn=<RsubBackward1>)
tensor([  4.9921,  -5.5637,   0.8049,  ...,   4.1643, -11.0824,  -5.4674],
       grad_fn=<DivBackward0>)
[Iter 529/659]  [sdf_loss: 7319.358398] [sdf: 1.166847] [g_hat_mean: 0.038566]
tensor(-0.1660, grad_fn=<RsubBackward1>)
tensor([ 3.3330,  1.3975, -0.6325,  ...,  3.8260, -3.3992, -0.7005],
       grad_fn=<DivBackward0>)
[Iter 530/659]  [sdf_loss: 9796.471680] [sdf: 1.537844] [g_hat_mean: 0.014909]
tensor(-0.5393, grad_fn=<RsubBackward1>)
tensor([ 0.9224,  0.9497, -0.6088,  ...,  0.7769,  0.0490, -7.1105],
       grad_fn=<DivBackward0>)
[Iter 531/659]  [sdf_loss: 2705.198242] [sdf: 0.765272] [g_hat_mean: 0.033297]
tensor(0.2347, grad_fn=<RsubBackward

tensor(0.1887, grad_fn=<RsubBackward1>)
tensor([-1.2966, -0.9152,  0.5344,  ...,  6.5805,  5.0271,  3.8849],
       grad_fn=<DivBackward0>)
[Iter 565/659]  [sdf_loss: 2801.214844] [sdf: 0.860607] [g_hat_mean: -0.004527]
tensor(0.1513, grad_fn=<RsubBackward1>)
tensor([ 0.7557,  3.8072, -1.5826,  ..., -2.1171, 34.8583,  3.7729],
       grad_fn=<DivBackward0>)
[Iter 566/659]  [sdf_loss: 850.011780] [sdf: 0.433890] [g_hat_mean: -0.009079]
tensor(0.5737, grad_fn=<RsubBackward1>)
tensor([-0.0373,  5.5972,  0.1723,  ...,  1.7704, -4.1832, -1.2328],
       grad_fn=<DivBackward0>)
[Iter 567/659]  [sdf_loss: 2711.529785] [sdf: 0.787496] [g_hat_mean: -0.027988]
tensor(0.2160, grad_fn=<RsubBackward1>)
tensor([ -0.4683,  16.2245,   2.0335,  ..., -10.4367,  -2.6325,  -1.3136],
       grad_fn=<DivBackward0>)
[Iter 568/659]  [sdf_loss: 383.320038] [sdf: 0.290061] [g_hat_mean: -0.022128]
tensor(0.7378, grad_fn=<RsubBackward1>)
tensor([-0.7220, -0.2396,  2.4180,  ..., -0.7647, -0.7053,  0.2356],
       

[Iter 602/659]  [sdf_loss: 561.693115] [sdf: 0.346043] [g_hat_mean: 0.020456]
tensor(0.6743, grad_fn=<RsubBackward1>)
tensor([ 6.7289e-02, -6.5170e-01, -1.6993e-01,  ...,  6.8211e+01,
        -1.5044e-01,  1.3972e+00], grad_fn=<DivBackward0>)
[Iter 603/659]  [sdf_loss: 2775.288574] [sdf: 0.633411] [g_hat_mean: -0.004581]
tensor(0.3799, grad_fn=<RsubBackward1>)
tensor([-0.3215, -0.2183,  0.4252,  ..., -7.3453, -0.3769,  1.8408],
       grad_fn=<DivBackward0>)
[Iter 604/659]  [sdf_loss: 4529.298828] [sdf: 1.005950] [g_hat_mean: -0.017158]
tensor(0.0165, grad_fn=<RsubBackward1>)
tensor([ 14.7322,  50.1053,  14.7143,  ..., -66.6462,  42.7416,  33.8406],
       grad_fn=<DivBackward0>)
[Iter 605/659]  [sdf_loss: 121.368317] [sdf: 0.166577] [g_hat_mean: -0.055192]
tensor(0.8769, grad_fn=<RsubBackward1>)
tensor([ 0.1425, -0.8308,  0.8667,  ...,  3.9358, -0.4794,  0.1884],
       grad_fn=<DivBackward0>)
[Iter 606/659]  [sdf_loss: 379.644806] [sdf: 0.166671] [g_hat_mean: 0.014451]
tensor(0.8354,

[Iter 640/659]  [sdf_loss: 13.469010] [sdf: 0.062189] [g_hat_mean: -0.005618]
tensor(0.9320, grad_fn=<RsubBackward1>)
tensor([ 0.4081, -0.7065,  0.0818,  ...,  0.3318, -0.3963,  1.1360],
       grad_fn=<DivBackward0>)
[Iter 641/659]  [sdf_loss: 9.089940] [sdf: 0.049757] [g_hat_mean: -0.032095]
tensor(0.9630, grad_fn=<RsubBackward1>)
tensor([-0.4567, -0.0880, -0.0606,  ..., -0.6029, -0.4307, -1.1393],
       grad_fn=<DivBackward0>)
[Iter 642/659]  [sdf_loss: 496.948730] [sdf: 0.373910] [g_hat_mean: -0.005686]
tensor(0.6271, grad_fn=<RsubBackward1>)
tensor([-0.2993, -0.9902,  0.3149,  ...,  0.0733, -0.0139, -0.2185],
       grad_fn=<DivBackward0>)
[Iter 643/659]  [sdf_loss: 5293.600098] [sdf: 0.982029] [g_hat_mean: 0.001837]
tensor(0.0218, grad_fn=<RsubBackward1>)
tensor([ 18.2827,  -8.8185, -18.9424,  ..., -38.4892, -70.0671, -33.9933],
       grad_fn=<DivBackward0>)
[Iter 644/659]  [sdf_loss: 4473.517090] [sdf: 1.004531] [g_hat_mean: -0.021784]
tensor(0.0021, grad_fn=<RsubBackward1>)
t